In [ ]:
##############################################################
### function for computing particle velocity distributions ###
##############################################################

def NEW_v(acc_1, acc_2, acc_3, acc1_length, acc2_length, acc3_length, velo, tof):

    if acc_2 != 0.0 and  acc2_length != 0.0:
        a2_term = (-np.sqrt(2.0*acc_1*acc1_length + velo**2) + 
                   np.sqrt(2.0*acc_1*acc1_length + 2.0*acc_2*acc2_length + velo**2))/acc_2    
    elif acc2_length != 0.0 and acc_2 == 0.0:
        a2_term = acc2_length/np.sqrt(2*acc_1*acc1_length + velo**2)    
    else:
        a2_term = 0.0

    if acc_3 != 0.0 and  acc3_length != 0.0:
        a3_term = (-np.sqrt(2.0*acc_1*acc1_length + 2.0*acc_2*acc2_length + velo**2) + 
                   np.sqrt(2.0*acc_1*acc1_length + 2.0*acc_2*acc2_length + 2.0*acc_3*acc3_length + velo**2))/acc_3   
    elif acc3_length != 0.0 and acc_3 == 0.0:
        a3_term = acc3_length/np.sqrt(2*acc_1*acc1_length + 2*acc_2*acc2_length + velo**2)
    else:
        a3_term = 0.0

    t_function = -tof + (-velo + np.sqrt(2.0*acc_1*acc1_length + velo**2))/acc_1 + a2_term + a3_term 
    v1 = velo*(1.01)

    if acc_2 != 0.0 and  acc2_length != 0.0:
        a2_term = (-np.sqrt(2.0*acc_1*acc1_length + v1**2) + 
                   np.sqrt(2.0*acc_1*acc1_length + 2.0*acc_2*acc2_length + v1**2))/acc_2    
    elif acc2_length != 0.0 and acc_2 == 0.0:
        a2_term = acc2_length/np.sqrt(2*acc_1*acc1_length + v1**2)   
    else:
        a2_term = 0.0

    if acc_3 != 0.0 and  acc3_length != 0.0:
        a3_term = (-np.sqrt(2.0*acc_1*acc1_length + 2.0*acc_2*acc2_length + v1**2) + 
                   np.sqrt(2.0*acc_1*acc1_length + 2.0*acc_2*acc2_length + 2.0*acc_3*acc3_length + v1**2))/acc_3    
    elif acc3_length != 0. and acc_3 == 0.0:
        a3_term = acc3_length/np.sqrt(2*acc_1*acc1_length + 2*acc_2*acc2_length + v1**2) 
    else:
        a3_term = 0.0

    Dt_function_dv = ( (-tof + (-v1 + np.sqrt(2.0*acc_1*acc1_length + v1**2)) / acc_1 
                        + a2_term + a3_term) - t_function ) / (v1 - velo) 
    return velo - t_function / Dt_function_dv

In [ ]:

def tof2mom_3accel(tof_ns, acc1mm, acc2mm, acc3mm, Efield1_Vpcm, Efield2_Vpcm, Efield3_Vpcm, charge_au, mass_amu):
    E_Field_au_over_si = 1.0 / 5.1421e11 #au * m / V
    length_au_over_m = 1.0 / 5.2917720859e-11 #au / m
    time_au_over_s = 1.0 / 2.418884326505e-17 #au / s
    AUmass_over_AMU = 1.660538782e-27 / 9.10938215e-31 #(au / kg) * (kg / amu)

    t = (tof_ns * 1e-9) * time_au_over_s
    mass = mass_amu * AUmass_over_AMU
    q = charge_au
    acc1_length = acc1mm * 0.001 * length_au_over_m
    acc2_length = acc2mm * 0.001 * length_au_over_m
    acc3_length = acc3mm * 0.001 * length_au_over_m
    Efield1_Vau = Efield1_Vpcm * 100.0 * E_Field_au_over_si
    Efield2_Vau = Efield2_Vpcm * 100.0 * E_Field_au_over_si
    Efield3_Vau = Efield3_Vpcm * 100.0 * E_Field_au_over_si

    acc1 = (Efield1_Vau * q) / mass
    acc2 = (Efield2_Vau * q) / mass
    acc3 = (Efield3_Vau * q) / mass

    acc1_temp = np.where(acc1 != 0, acc1, 0.001)
    acc2_temp = np.where(acc2 != 0, acc2, 0.001)
    acc3_temp = np.where(acc3 != 0, acc3, 0.001)

    v = np.sqrt(2.0) * np.sqrt(acc1_temp * acc1_length) - \
        (np.sqrt(2.0) * acc1_temp * np.sqrt(acc1_temp * acc1_length) / acc2_temp) + \
        (np.sqrt(2.0) * acc1_temp * np.sqrt(acc1_temp * acc1_length + acc2_temp * acc2_length) / acc2_temp) - \
        (np.sqrt(2.0) * acc1_temp * np.sqrt(acc1_temp * acc1_length + acc2_temp * acc2_length) / acc3_temp) + \
        (np.sqrt(2.0) * acc1_temp * np.sqrt(acc1_temp * acc1_length + acc2_temp * acc2_length + acc3_temp * acc3_length) / acc3_temp) - \
        acc1_temp * t

    for i in range(len(tof_ns)):
        new_v = 0.0
        shift_counter = 1
        for j in range(50):
            new_v = NEW_v(acc1, acc2, acc3, acc1_length, acc2_length, acc3_length, v[i], t[i])

            if np.isinf(new_v) == 1:
                new_v = v[i] * (1.0 + 2.0 ** shift_counter / 1000.0)
                shift_counter += 1
            else:
                shift_counter = 1

            if np.abs((new_v * mass - v[i] * mass) / (v[i] * mass)) < 1e-6:
                break

            v[i] = new_v

    p_z = v * mass
    return p_z


In [ ]:
def calc_px(tof_ns, x_mm, y_mm, mass_amu, charge_au, BField_ns, BField_clockwise=True):
    MEKG = 9.10938356e-31  # kg
    COULOMB = 1.602176634e-19  # C
    MASSAU = 1.66053906660e-27  # kg
    time_au_over_s = 1.0 / 2.418884326505e-17 #au / s
    dist_au_over_m = 1.0 / 5.29177210903e-11 # au / m
    mass_au_over_amu = 1.660538782e-27 / 9.10938215e-31 #(au / kg) * (kg / amu)
 
    
    #convert all to a.u.
    tof_au = tof_ns * 1e-9 * time_au_over_s
    x_au = x_mm * 1e-3 * dist_au_over_m
    y_au = y_mm * 1e-3 * dist_au_over_m
    mass_au = mass_amu * mass_au_over_amu

    if mass_amu < 1.0:
        w, a, b = 0.0, 0.0, 0.0

        m = mass_amu * MASSAU * MEKG
        q = charge_au * COULOMB
        pau = m * 300e6 / 137.

        fieldB = 2. * m * 3.14152 / (q * BField_ns * 1e-9)

        if BField_clockwise:
            fieldB = -fieldB

        w = q / m * fieldB
        a = (1. - np.cos(w * tof_ns * 1e-9)) / w
        b = (np.sin(w * tof_ns * 1e-9)) / w

        px = m * (x_mm / 1000. * b + a * y_mm / 1000.) / (a * a + b * b)
        px = px / pau
    else:
        #vau = 2.1877e+6
        px = x_au / tof_au * mass_au

    return px


In [ ]:
def calc_py(tof_ns, x_mm, y_mm, mass_amu, charge_au, BField_ns, BField_clockwise=True):
    MEKG = 9.10938356e-31  # kg
    COULOMB = 1.602176634e-19  # C
    MASSAU = 1.66053906660e-27  # kg
    time_au_over_s = 1.0 / 2.418884326505e-17 #au / s
    dist_au_over_m = 1.0 / 5.29177210903e-11 # au / m
    mass_au_over_amu = 1.660538782e-27 / 9.10938215e-31 #(au / kg) * (kg / amu)
 
    #convert all to a.u.
    tof_au = tof_ns * 1e-9 * time_au_over_s
    x_au = x_mm * 1e-3 * dist_au_over_m
    y_au = y_mm * 1e-3 * dist_au_over_m
    mass_au = mass_amu * mass_au_over_amu
    
    if mass_amu < 1.0:
        w, a, b = 0.0, 0.0, 0.0

        m = mass_amu * MASSAU * MEKG
        q = charge_au * COULOMB
        pau = m * 300e6 / 137.

        fieldB = 2. * m * 3.14152 / (q * BField_ns * 1e-9)

        if BField_clockwise:
            fieldB = -fieldB

        w = q / m * fieldB
        a = (1. - np.cos(w * tof_ns * 1e-9)) / w
        b = (np.sin(w * tof_ns * 1e-9)) / w

        py = m * (-x_mm / 1000. * a + b * y_mm / 1000.) / (a * a + b * b)

        py = py / pau
    else:
        py = y_au / tof_au * mass_au

    return py


In [ ]:
def calc_pze(tof_ns, x_mm, y_mm, mass_amu, charge_au, para_a, para_b, para_c, acc1mm, acc2mm, acc3mm, Efield1_Vpcm, Efield2_Vpcm, Efield3_Vpcm, BField_ns, BField_clockwise=True):
    px, py, pz, E1 = 0, 0, 0, 0

    if mass_amu < 1.:
        m = mass_amu * 1.66053906660e-27  # Convert mass from atomic mass units to kg
        q = -charge_au * 1.602176634e-19  # Convert charge from atomic units to coulombs
        pau = m * 300e6 / 137.036  # Atomic unit of momentum

        fieldB = 2. * m * np.pi / (q * BField_ns * 1e-9)

        if BField_clockwise:
            fieldB = -fieldB

        w = q / m * fieldB
        a = (1. - np.cos(w * tof_ns * 1.e-9)) / w
        b = (np.sin(w * tof_ns * 1.e-9)) / w

        px = m * (x_mm / 1000. * b + a * y_mm / 1000.) / (a**2 + b**2)
        px /= pau
        py = m * (-x_mm / 1000. * a + b * y_mm / 1000.) / (a**2 + b**2)
        py /= pau

        q = -charge_au
        m = mass_amu
        E1 = Efield1_Vpcm
        pz = -tof2mom_3accel(tof_ns, acc1mm, acc2mm, acc3mm, E1, Efield2_Vpcm, Efield3_Vpcm, q, m)
        
#         for i in range(50):
#             if np.abs(para_a - para_b * np.power(para_c, (px**2 + py**2 + pz**2) * pau**2 / (2 * m[:,0] * q)) - E1) > 1e-4:
#                 E1 = para_a - para_b * np.power(para_c, (px**2 + py**2 + pz**2) * pau**2 / (2 * m * q))
#                 pz = -tof2mom_3accel(tof_ns, acc1mm, acc2mm, acc3mm, E1, Efield2_Vpcm, Efield3_Vpcm, q, m)

    return pz